- Import necessary libraries: pandas for data manipulation, numpy for numerical operations, re for regular expressions, and os for operating system related functionalities.

In [57]:
import pandas as pd 
import os
import seaborn as sns

1. Reading and combining data
   - Load all 9 CSV files into a list.

In [58]:
folder_path= './PassengerVehicle_Stats'
dfs=[]
files= [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in files:
    file_path = os.path.join(folder_path,file)
    df1 = pd.read_csv(file_path)
    dfs.append(df1)

- Concatenate the files into a single DataFrame, named vehicles_df.

In [59]:
combine_df = pd.concat(dfs, ignore_index=True)
combine_df.to_csv('./vehicles_df.csv', index= False)

vehicles_df = pd.read_csv('vehicles_df.csv')



2. Initial data exploration and cleaning
    - Examine the DataFrame structure, including its features and data types.

In [60]:
print(vehicles_df.info())
print(vehicles_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16602 entries, 0 to 16601
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          16602 non-null  int64  
 1   Public Vehicle Number               16602 non-null  int64  
 2   Status                              16602 non-null  object 
 3   Vehicle Make                        14685 non-null  object 
 4   Vehicle Model                       14639 non-null  object 
 5   Vehicle Model Year                  14660 non-null  float64
 6   Vehicle Color                       14616 non-null  object 
 7   Vehicle Fuel Source                 16602 non-null  object 
 8   Wheelchair Accessible               16602 non-null  object 
 9   Company Name                        16602 non-null  object 
 10  Address                             14816 non-null  object 
 11  City                                14816

- Examine the DataFrame structure.

In [61]:
vehicles_df.shape

(16602, 17)

- Remove any duplicate records.

In [62]:
vehicles_df_cleaned = vehicles_df.drop_duplicates(subset='Record ID', keep='first')

- Examine the DataFrame structure after removing the duplicates.

In [63]:
vehicles_df_cleaned.shape

(15667, 17)

- Checking for null values for each of the columns.

In [64]:
vehicles_df_cleaned.isnull().sum()

Unnamed: 0                               0
Public Vehicle Number                    0
Status                                   0
Vehicle Make                          1816
Vehicle Model                         1859
Vehicle Model Year                    1841
Vehicle Color                         1883
Vehicle Fuel Source                      0
Wheelchair Accessible                    0
Company Name                             0
Address                               1694
City                                  1694
State                                 1694
ZIP Code                              1694
Taxi Affiliation                      8710
Taxi Medallion License Management     8736
Record ID                                0
dtype: int64

- Remove null records for the 'Address', 'city', 'state', and 'ZIP Code' columns. 
  - Reasons for removing null records - There are same number of entries are missing in these 4 columns and those are in same entries.

In [65]:
columns_to_check = ['Address', 'City', 'State','ZIP Code']

vehicles_df_cleaned.dropna(subset=columns_to_check, inplace=True)


C:\Users\DELL\AppData\Local\Temp\ipykernel_17816\2284951951.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicles_df_cleaned.dropna(subset=columns_to_check, inplace=True)


- Checking for null values for each column after filtering.

In [66]:
vehicles_df_cleaned.isnull().sum()

Unnamed: 0                               0
Public Vehicle Number                    0
Status                                   0
Vehicle Make                           128
Vehicle Model                          171
Vehicle Model Year                     153
Vehicle Color                          195
Vehicle Fuel Source                      0
Wheelchair Accessible                    0
Company Name                             0
Address                                  0
City                                     0
State                                    0
ZIP Code                                 0
Taxi Affiliation                      7016
Taxi Medallion License Management     7042
Record ID                                0
dtype: int64

- Filling NaN values in 'Vehicle Model Year' with 0 and convert the column to integer type.

In [67]:
vehicles_df_cleaned['Vehicle Model Year'].fillna(0, inplace = True)
vehicles_df_cleaned['Vehicle Model Year']=vehicles_df_cleaned['Vehicle Model Year'].astype(int)


C:\Users\DELL\AppData\Local\Temp\ipykernel_17816\316356643.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  vehicles_df_cleaned['Vehicle Model Year'].fillna(0, inplace = True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17816\316356643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicles_df_cleaned['Vehicle Model Year'].fillna(0, inp

- Filling NaN values in the DataFrame with "unknown".

In [68]:
vehicles_df_cleaned.fillna("unknown", inplace = True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17816\4265509047.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicles_df_cleaned.fillna("unknown", inplace = True)


- Descriptive statistics of the 'Vehicle Model Year' column after cleaning.


In [69]:
vehicles_df_cleaned['Vehicle Model Year'].describe()

count    13973.000000
mean      1990.866671
std        224.386734
min          0.000000
25%       2013.000000
50%       2016.000000
75%       2021.000000
max       2025.000000
Name: Vehicle Model Year, dtype: float64

- Checking outliers of the 'Vehicle Model Year' column.

In [70]:
Q1 = vehicles_df_cleaned['Vehicle Model Year'].quantile(0.25)
Q3 = vehicles_df_cleaned['Vehicle Model Year'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = vehicles_df_cleaned[(vehicles_df_cleaned['Vehicle Model Year'] < lower_bound) | (vehicles_df_cleaned['Vehicle Model Year'] > upper_bound)]

print("Outliers:\n", outliers)

df['Vehicle Model Year'].value_counts()

Outliers:
        Unnamed: 0  Public Vehicle Number     Status  Vehicle Make  \
106           584                  50000   INACTIVE  FREIGHTLINER   
107           881                  15065   INACTIVE      CHAMPION   
109          1422                1000009   INACTIVE      CHAMPION   
114          2457                1000013   INACTIVE      CHAMPION   
118          3026                  13754   INACTIVE          INTL   
...           ...                    ...        ...           ...   
16562       14215                     75  VIOLATION    MAINSTREET   
16573       14920                     66  VIOLATION    MAINSTREET   
16585       15509                     68  VIOLATION    MAINSTREET   
16588       15738                    102  SURRENDER         TIPKE   
16599       16216                     70  VIOLATION    MAINSTREET   

        Vehicle Model  Vehicle Model Year Vehicle Color Vehicle Fuel Source  \
106    X LINE SHUTTLE                2000         GREEN              Diesel   
10

Vehicle Model Year
2014    1444
2023    1439
2013    1273
2012    1142
2022    1051
2019     794
2011     770
2015     768
2016     744
2024     734
2020     599
2021     571
2017     546
2018     533
2010     309
2008     182
2006     175
2009     124
2007     122
2005      75
2004      60
2001      60
2025      53
2003      40
2002      36
1999      35
1997      26
2000      21
1998      14
1996       9
1984       7
1995       6
1991       5
1988       5
1992       4
1981       3
1990       3
1994       3
1993       3
1986       2
1989       2
1985       2
1983       1
1987       1
1980       1
Name: count, dtype: int64

-  Filtering out rows where 'Vehicle Model Year' is greater than 1900 and checking the shape of the resulting DataFrame.


In [71]:

filtered_df_cleaned = vehicles_df_cleaned[vehicles_df_cleaned['Vehicle Model Year']> 1900]
filtered_df_cleaned.shape

(13797, 17)

- Descriptive statistics of the 'Vehicle Model Year' column after removing outliers.


In [72]:
filtered_df_cleaned['Vehicle Model Year'].describe()

count    13797.000000
mean      2016.175763
std          5.510874
min       1980.000000
25%       2013.000000
50%       2016.000000
75%       2021.000000
max       2025.000000
Name: Vehicle Model Year, dtype: float64

- Adding a new column 'Vehicle Type' to the Dataframe.

In [73]:
filtered_df_cleaned['Vehicle Type'] = filtered_df_cleaned['Record ID'].str.extract(r'(\D+)$')[0]

filtered_df_cleaned.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_17816\3549340613.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_cleaned['Vehicle Type'] = filtered_df_cleaned['Record ID'].str.extract(r'(\D+)$')[0]


,Unnamed: 0,Public Vehicle Number,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,Address,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID,Vehicle Type
0,1286,12009,RESERVED,CHEVROLET,EXPRESS,2014,BLACK,Bio-Diesel,N,CHICAGO PRIVATE TOURS LLC,4567 S. OAKENWALD AVE.,CHICAGO,IL,60653.0,unknown,unknown,12009Charter Sightseeing,Charter Sightseeing
1,2095,12248,INACTIVE,MERCEDES,SPRINTER,2010,SILVER,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,4610 N. CLARK ST.,CHICAGO,IL,60640.0,unknown,unknown,12248Charter Sightseeing,Charter Sightseeing
2,7950,13527,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",4400 S. RACINE AVE.,CHICAGO,IL,60609.0,unknown,unknown,13527Charter Sightseeing,Charter Sightseeing
4,9359,13528,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",4400 S. RACINE AVE.,CHICAGO,IL,60609.0,unknown,unknown,13528Charter Sightseeing,Charter Sightseeing
5,9441,12025,INACTIVE,MERCEDES,SPRINTER,2015,BLACK,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,4610 N. CLARK ST.,CHICAGO,IL,60640.0,unknown,unknown,12025Charter Sightseeing,Charter Sightseeing


- Droping the columns “Address” and “Public Vehicle Number”.

In [74]:
filtered_df_cleaned.drop(columns=['Address', 'Public Vehicle Number'], inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17816\1594787936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_cleaned.drop(columns=['Address', 'Public Vehicle Number'], inplace=True)


In [75]:
filtered_df_cleaned.head()

,Unnamed: 0,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID,Vehicle Type
0,1286,RESERVED,CHEVROLET,EXPRESS,2014,BLACK,Bio-Diesel,N,CHICAGO PRIVATE TOURS LLC,CHICAGO,IL,60653.0,unknown,unknown,12009Charter Sightseeing,Charter Sightseeing
1,2095,INACTIVE,MERCEDES,SPRINTER,2010,SILVER,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,CHICAGO,IL,60640.0,unknown,unknown,12248Charter Sightseeing,Charter Sightseeing
2,7950,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",CHICAGO,IL,60609.0,unknown,unknown,13527Charter Sightseeing,Charter Sightseeing
4,9359,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",CHICAGO,IL,60609.0,unknown,unknown,13528Charter Sightseeing,Charter Sightseeing
5,9441,INACTIVE,MERCEDES,SPRINTER,2015,BLACK,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,CHICAGO,IL,60640.0,unknown,unknown,12025Charter Sightseeing,Charter Sightseeing


In [76]:
filtered_df_cleaned.shape

(13797, 16)

Taking filtered_df_cleaned as df dataframe.

In [77]:
df = filtered_df_cleaned

In [78]:
df.head()

,Unnamed: 0,Status,Vehicle Make,Vehicle Model,Vehicle Model Year,Vehicle Color,Vehicle Fuel Source,Wheelchair Accessible,Company Name,City,State,ZIP Code,Taxi Affiliation,Taxi Medallion License Management,Record ID,Vehicle Type
0,1286,RESERVED,CHEVROLET,EXPRESS,2014,BLACK,Bio-Diesel,N,CHICAGO PRIVATE TOURS LLC,CHICAGO,IL,60653.0,unknown,unknown,12009Charter Sightseeing,Charter Sightseeing
1,2095,INACTIVE,MERCEDES,SPRINTER,2010,SILVER,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,CHICAGO,IL,60640.0,unknown,unknown,12248Charter Sightseeing,Charter Sightseeing
2,7950,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",CHICAGO,IL,60609.0,unknown,unknown,13527Charter Sightseeing,Charter Sightseeing
4,9359,INACTIVE,VAN HOOL,TD925,2008,RED,Bio-Diesel,N,"TRT TRANSPORTATION, INC.",CHICAGO,IL,60609.0,unknown,unknown,13528Charter Sightseeing,Charter Sightseeing
5,9441,INACTIVE,MERCEDES,SPRINTER,2015,BLACK,Bio-Diesel,N,O'HARE-MIDWAY LIMOUSINE SERVICE INC # 2,CHICAGO,IL,60640.0,unknown,unknown,12025Charter Sightseeing,Charter Sightseeing


Apply valid city names to 'City' column.

In [79]:
df['City'].unique()

array(['CHICAGO', 'ELMHURST', 'ORLAND PARK', 'ST. CHARLES', 'SKOKIE',
       'FRANKFORT', 'ELGIN', 'SHOREWOOD', 'TINLEY PARK', 'LYNWOOD',
       'CHIICAGO', 'LOVES PARK', 'BARRINGTON', 'NEW LENOX', 'DOLTON',
       'ELK GROVE VILLAGE', 'CHICAGO RIDGE', 'BARTLETT', 'EVERGREEN PARK',
       'ARLINGTON HEIGHTS', 'BROOKFIELD', 'EARLVILLE', 'SCHILLER PARK',
       'CHGO', 'PLAINFIELD', 'DEERFIELD', 'OAKBROOK', 'NORTHBROOK',
       'BENSENVILLE', 'JOHNSBURG', 'NORRIDGE', 'DESPLAINES',
       'DES PLAINES', 'ITASCA', 'CHCAGO', 'GURNEE', 'METTAWA'],
      dtype=object)

In [80]:
valid_cities = [
    'CHICAGO', 'ELMHURST', 'ORLAND PARK', 'ST. CHARLES', 'SKOKIE', 'FRANKFORT', 'ELGIN', 'SHOREWOOD', 
    'TINLEY PARK', 'LYNWOOD', 'LOVES PARK', 'BARRINGTON', 'NEW LENOX', 'DOLTON', 'CHICAGO RIDGE', 
    'BARTLETT', 'EVERGREEN PARK', 'ARLINGTON HEIGHTS', 'BROOKFIELD', 'EARLVILLE', 'SCHILLER PARK', 
    'PLAINFIELD', 'DEERFIELD', 'OAK BROOK', 'NORTHBROOK', 'BENSENVILLE', 'ELK GROVE VILLAGE', 'JOHNSBURG', 
    'NORRIDGE', 'DES PLAINES', 'ITASCA', 'GURNEE', 'METTAWA'
]

In [81]:
def correct_city_name(city):
    
    city_corrections = {
        'CHIICAGO': 'CHICAGO',
        'CHGO': 'CHICAGO',
        'OAKBROOK': 'OAK BROOK',
        'DESPLAINES': 'DES PLAINES',
        'CHCAGO': 'CHICAGO'
    }
  
    return city_corrections.get(city, city)

In [82]:
df['City'] = df['City'].apply(correct_city_name)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17816\1089565626.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['City'] = df['City'].apply(correct_city_name)


In [83]:
df['City'].unique()

array(['CHICAGO', 'ELMHURST', 'ORLAND PARK', 'ST. CHARLES', 'SKOKIE',
       'FRANKFORT', 'ELGIN', 'SHOREWOOD', 'TINLEY PARK', 'LYNWOOD',
       'LOVES PARK', 'BARRINGTON', 'NEW LENOX', 'DOLTON',
       'ELK GROVE VILLAGE', 'CHICAGO RIDGE', 'BARTLETT', 'EVERGREEN PARK',
       'ARLINGTON HEIGHTS', 'BROOKFIELD', 'EARLVILLE', 'SCHILLER PARK',
       'PLAINFIELD', 'DEERFIELD', 'OAK BROOK', 'NORTHBROOK',
       'BENSENVILLE', 'JOHNSBURG', 'NORRIDGE', 'DES PLAINES', 'ITASCA',
       'GURNEE', 'METTAWA'], dtype=object)

In [84]:
df = df[df['City'].isin(valid_cities)]

In [85]:
df['City'].unique()

array(['CHICAGO', 'ELMHURST', 'ORLAND PARK', 'ST. CHARLES', 'SKOKIE',
       'FRANKFORT', 'ELGIN', 'SHOREWOOD', 'TINLEY PARK', 'LYNWOOD',
       'LOVES PARK', 'BARRINGTON', 'NEW LENOX', 'DOLTON',
       'ELK GROVE VILLAGE', 'CHICAGO RIDGE', 'BARTLETT', 'EVERGREEN PARK',
       'ARLINGTON HEIGHTS', 'BROOKFIELD', 'EARLVILLE', 'SCHILLER PARK',
       'PLAINFIELD', 'DEERFIELD', 'OAK BROOK', 'NORTHBROOK',
       'BENSENVILLE', 'JOHNSBURG', 'NORRIDGE', 'DES PLAINES', 'ITASCA',
       'GURNEE', 'METTAWA'], dtype=object)

In [86]:
city_coordinates = {
    'CHICAGO': (41.8781, -87.6298),
    'ELMHURST': (41.8991, -87.9403),
    'ORLAND PARK': (41.6295, -87.8534),
    'ST. CHARLES': (41.9140, -88.3052),
    'SKOKIE': (42.0324, -87.7411),
    'FRANKFORT': (41.3102, -87.8506),
    'ELGIN': (42.0354, -88.2812),
    'SHOREWOOD': (41.7799, -88.2281),
    'TINLEY PARK': (41.5742, -87.7820),
    'LYNWOOD': (41.5742, -87.6232),
    'LOVES PARK': (42.3019, -89.0543),
    'BARRINGTON': (42.1531, -88.1382),
    'NEW LENOX': (41.5130, -87.9812),
    'DOLTON': (41.6190, -87.5936),
    'CHICAGO RIDGE': (41.7150, -87.7820),
    'BARTLETT': (41.9951, -88.1851),
    'EVERGREEN PARK': (41.7107, -87.7893),
    'ARLINGTON HEIGHTS': (42.0884, -87.9806),
    'BROOKFIELD': (41.8314, -87.7846),
    'EARLVILLE': (41.6274, -88.6947),
    'SCHILLER PARK': (41.9781, -87.8182),
    'PLAINFIELD': (41.6303, -88.2039),
    'DEERFIELD': (42.1546, -87.8472),
    'OAK BROOK': (41.8503, -87.9511),
    'NORTHBROOK': (42.1211, -87.8294),
    'BENSENVILLE': (41.9783, -87.9401),
    'ELK GROVE VILLAGE': (42.0353, -87.9732),
    'JOHNSBURG': (42.3191, -88.2733),
    'NORRIDGE': (41.9810, -87.7947),
    'DES PLAINES': (42.0396, -87.8704),
    'ITASCA': (41.9534, -88.0169),
    'GURNEE': (42.3620, -87.8837),
    'METTAWA': (42.2573, -87.9116)
}

In [87]:
df['Latitude'] = df['City'].map(lambda city: city_coordinates.get(city, (None, None))[0])
df['Longitude'] = df['City'].map(lambda city: city_coordinates.get(city, (None, None))[1])

In [88]:
df['City'].unique()

array(['CHICAGO', 'ELMHURST', 'ORLAND PARK', 'ST. CHARLES', 'SKOKIE',
       'FRANKFORT', 'ELGIN', 'SHOREWOOD', 'TINLEY PARK', 'LYNWOOD',
       'LOVES PARK', 'BARRINGTON', 'NEW LENOX', 'DOLTON',
       'ELK GROVE VILLAGE', 'CHICAGO RIDGE', 'BARTLETT', 'EVERGREEN PARK',
       'ARLINGTON HEIGHTS', 'BROOKFIELD', 'EARLVILLE', 'SCHILLER PARK',
       'PLAINFIELD', 'DEERFIELD', 'OAK BROOK', 'NORTHBROOK',
       'BENSENVILLE', 'JOHNSBURG', 'NORRIDGE', 'DES PLAINES', 'ITASCA',
       'GURNEE', 'METTAWA'], dtype=object)

- Note: cleaned dataframe is 'df'.